#Sequential Scan Vs Rtree method 

## Getting Started 

### Cloning from my own github repository to gain access to file provided by tutor.

In [1]:
!git clone https://github.com/snj-adhikari/r_tree_sample.git

Cloning into 'r_tree_sample'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


 Just changing the directory to currently cloned directory

In [2]:
%cd /content/r_tree_sample

/content/r_tree_sample


### Creating dataframe from dataset.txt

We were provided with dataset.txt file for our assignment.
Here , the first line of the file provides the no of dataset present. so , we used first line to get total no of dataset and following lines where used to create dataset DataFrame. 

Each line of text file was splitted , getting id , x_cord , y_cord from each line. 
So index column of dataframe was replaced using **id** got from the each line and two columns named x_cord , y_cord was made on the dataframe.

In [4]:
import re
import pandas as pd

dataFile = 'dataset_r.txt'
dataset_list = []
no_of_data =0
count = 0
with open(dataFile) as f:
    for line in f:
        # remove whitespace at the start and the newline at the end
        line = line.strip()
        if(count == 0):
          no_of_data = line.strip()
        else :
          # split each column on whitespace
          columns = re.split('\s+', line)
          dataset_list.append(columns)
#           print(columns)
        
        count +=1

dataframe_of_datalist =  pd.DataFrame(dataset_list , columns=['id' , 'x_cod' , 'y_cod'])

dataframe_of_datalist = dataframe_of_datalist.apply(pd.to_numeric)
dataframe_of_datalist.set_index('id', inplace=True)
dataframe_of_datalist.head()

,x_cod,y_cod
id,,
1,8224,78217
2,68940,46095
3,92607,82845
4,59,99459
5,14140,65521


### Creating queries DataFrame from queries.txt

here , we have read queris.txt using Open function of python. Then created array based on spliting space on each line of the file. Since , we need co-ordinate x_1 , y_1 , x_2 , y_2 for our rtree package to get point boundaries. The default file provide the structure of x_1 , x_2 , y_1 , y_2  , we had to re-arrange the array converted from spliting the each line to format we needed.

Hence , the resulting dataframe has column x_1 , y_1 , x_2 , y_2 along with index.

In [5]:
import re
import pandas as pd

queryFile = 'queries.txt'
queries_list = []
no_of_data =0
count = 0
with open(queryFile) as f:
    for line in f:
        # remove whitespace at the start and the newline at the end
        line = line.strip()
        
        # split each column on whitespace
        columns = re.split('\s+', line)
        re_index = [columns[0] , columns[2]  , columns[1] , columns[3]]
        queries_list.append(re_index)
#           print(columns)
        
        count +=1

dataframe_of_queries =  pd.DataFrame(queries_list , columns=['x_1' , 'y_1' , 'x_2', 'y_2'])

dataframe_of_queries = dataframe_of_queries.apply(pd.to_numeric)
dataframe_of_queries.head()

,x_1,y_1,x_2,y_2
0,17840,13971,18840,14971
1,33451,29693,34451,30693
2,791,2515,1791,3515
3,81921,94973,82921,95973
4,75678,53545,76678,54545


## Lets use Sequential Scan based on panda DataFrame 

This function takes two parameter **data** and **queries** which is both DataFrame type.
This is the manaul  function , we loop through queries dataframe and use dataframe default **loc** function to compare dataset values on dataset dataframe passed on the function to find the resulting points between the queries co-ordinates. Finally , return the execution time taken by the sequential scan based  method to get result on 100 queries.

Pandas DataFrame.loc attribute access a group of rows and columns by label(s) or a boolean array in the given DataFrame. On our case each cell is scanned by the function based on our condition and return the cell that satisfies the result.

In [0]:
import time
def gen_sequence_method(data , queries):
  start_seq = time.process_time()
  test_count = []
  # for index,item in enumerate(test):
  for i,query in queries.iterrows() : 
    query_result = dataframe_of_datalist.loc[((dataframe_of_datalist['x_cod'] >= query['x_1']) & (dataframe_of_datalist['x_cod'] <= query['x_2'])) & ((dataframe_of_datalist['y_cod'] >= query['x_1']) & (dataframe_of_datalist['y_cod'] <= query['x_2']))]
    test_count.append(len(query_result.index))
  end_seq = time.process_time()

  time_expand = end_seq - start_seq
  print("Sequential Query start")
  print("******************************************")
  print("Points found :: " , test_count)
  print( "Total Time taken :: " , time_expand)
  print("Average Time Taken by each query :: ",time_expand/100 )
  print("******************************************")

  return time_expand
    


#R-Tree Based Method

Installing rtree and necessary libraries

In [7]:
!pip install rtree
!sudo apt install libspatialindex-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 8 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 kB]
Fetched 555 kB in 1s (373 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.p

This function takes two parameter **data** and **queries** which is both DataFrame type.
This is the rtee package based function , we create index from given dataset passed on the function and used passed queries to find the resulting points between the queries co-ordinates. Finally , return the execution time taken by the rtree method to get result on 100 queries.

In [0]:
from rtree import index 

def using_rtree_method(data , queries) :
  idx = index.Index()
  index_start = time.process_time()
  for i, points in data.iterrows() : 
    idx.insert(i, points.to_list())
  

  query_start = time.process_time()
  test_count = []
  for i,query in queries.iterrows() : 
    # print(query.to_list())
    result_points = list(idx.intersection(query.to_list()))
    test_count.append(len(result_points))
  
  query_end = time.process_time()
  query_time = query_end - query_start
  print("R tree based method start")
  print('_________________________________________')
  print('Number of points :', test_count)
  print('Time taken for 100 queries :',query_time)
  print('Average time taken by single query :' , query_time/100)

  return query_time




## Nows Lets Compare result between this two method

In [9]:
seq_time = gen_sequence_method(dataframe_of_datalist , dataframe_of_queries)
rtree_time = using_rtree_method(dataframe_of_datalist , dataframe_of_queries)

print("******************************************")
print("Comparing the time taken by both \n lets c what happens" , seq_time/rtree_time)

Sequential Query start
******************************************
Points found ::  [7, 10, 10, 17, 10, 8, 13, 7, 10, 14, 9, 9, 10, 14, 11, 7, 12, 6, 11, 11, 8, 9, 10, 13, 11, 11, 8, 10, 5, 8, 13, 10, 7, 12, 9, 10, 8, 6, 7, 10, 6, 13, 14, 9, 14, 15, 9, 16, 9, 11, 13, 14, 11, 20, 12, 9, 4, 10, 8, 15, 17, 11, 10, 6, 11, 14, 10, 10, 6, 9, 13, 10, 8, 8, 8, 7, 8, 15, 9, 9, 7, 11, 16, 10, 8, 8, 7, 5, 7, 12, 8, 9, 11, 11, 11, 13, 11, 19, 13, 11]
Total Time taken ::  0.38038798200000024
Average Time Taken by each query ::  0.0038038798200000024
******************************************
R tree based method start
_________________________________________
Number of points : [15, 13, 9, 8, 8, 8, 12, 10, 9, 11, 11, 6, 6, 10, 6, 8, 10, 10, 11, 7, 11, 9, 4, 8, 11, 12, 7, 13, 9, 9, 8, 7, 9, 9, 10, 13, 9, 6, 8, 13, 12, 8, 8, 11, 11, 10, 8, 8, 12, 5, 7, 4, 6, 13, 9, 9, 7, 10, 16, 4, 11, 6, 6, 16, 14, 8, 11, 10, 7, 16, 13, 11, 10, 7, 10, 9, 10, 11, 6, 11, 10, 8, 9, 10, 20, 8, 8, 12, 7, 9, 5, 14, 11, 5, 6